_Aboa Bouadou, Rose Guionnet, Nathan Laîné, Victor Devys, Tino Margely_

## Import Modules

In [1]:
import pandas as pd
from datetime import datetime, timedelta

In [2]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import math
import networkx as nx
from networkx.drawing.nx_pydot import graphviz_layout
from matplotlib import pyplot as plt
import copy
import time
from pulp import *

In [3]:
SECONDS_PER_DAY = 24*60*60

## Read Data

In [4]:
data = pd.read_excel("data/raw/Orano-données.xlsx", sheet_name="Optim", header=5, usecols=[3,4,5,6,7], nrows=47)

In [5]:
data.head()

,Commandes,ordre,disp,max,durée
0,c1,1,2021-01-01,2021-06-01,6.057056
1,c2,2,2021-01-02,2021-06-02,59.133726
2,c3,3,2021-01-03,2021-06-03,18.738048
3,c4,4,2021-01-04,2021-06-04,6.044729
4,c5,5,2021-01-05,2021-06-05,71.008938


In [6]:
data['disp'].min().to_pydatetime()

datetime.datetime(2021, 1, 1, 0, 0)

# Convert Excel to problem data

In [7]:
def dataframe_to_list(data):
    dataframe = data.copy()
    min_date = dataframe['disp'].min().to_pydatetime()
    dataframe['disp'] = dataframe['disp'].apply(lambda x: (x.to_pydatetime()- min_date).total_seconds()/SECONDS_PER_DAY)
    dataframe['max'] = dataframe['max'].apply(lambda x: (x.to_pydatetime()-min_date).total_seconds()/SECONDS_PER_DAY)
    dataframe['durée'] = dataframe['durée']/15
    
    return(list(dataframe['disp']),list(dataframe['max']),list(dataframe['durée']))

In [8]:
disp, dmax, duree = dataframe_to_list(data)
disp, dmax, duree = disp[:15], dmax[:15], duree[:15]

In [9]:
print("Disp : " , disp[:5], "...")
print("Max : " , dmax[:5], "...")
print("Durée : " , duree[:5], "...")

Disp :  [0.0, 1.0, 2.0, 3.0, 4.0] ...
Max :  [151.0, 152.0, 153.0, 154.0, 155.0] ...
Durée :  [0.4038037609211829, 3.9422484045068145, 1.2492032181415056, 0.4029819404789006, 4.7339292113824065] ...


In [10]:
def isFeasible(disp, dmax, duree):
    time = sum(duree)
    date_max_max = max(dmax)
    print("time sum : ", time)
    print("Date Max max : ", date_max_max)

In [11]:
isFeasible(disp, dmax, duree)

time sum :  48.62963214409862
Date Max max :  165.0


## Première résolution

In [12]:
def solvePLNE(disp, dmax, duree):
    
    Tasks=range(len(disp)) #Liste des taches 
    
    model = LpProblem("Orano", LpMaximize)
    
    # Variables 
    x = LpVariable.dicts("x", ((i, j) for i in Tasks for j in Tasks),0,1,LpInteger) #x_{i,j}=1 si i est usinée avant j
    deb = LpVariable.dicts("deb", Tasks,0, None)
    dem = LpVariable.dicts("dem", Tasks,0, None)
    real = LpVariable.dicts("real", Tasks,0, None)
    marge = LpVariable.dicts("marge", Tasks,0, None)
    liv = LpVariable.dicts("liv", Tasks,0, None)
    z = LpVariable("z", lowBound=0)
    
    # Contraintes :
    for j in Tasks:
        model += dem[j] >= disp[j]
        model += real[j] == dem[j] + duree[j] 
        model += dem[j] == lpSum(duree[i]*x[i,j] for i in Tasks)
        model += liv[j] == deb[j] + 30
        model += liv[j] <= dmax[j]
        model += marge[j] == dmax[j] - liv[j]
        model += deb[j] >= real[j]
        model += z <= marge[j]
        for i in Tasks:
            model += x[j,i] + x[i,j] == 1 
            for k in Tasks:
                model += x[i,k] >= x[i,j] + x[j,k] - 1
    
    
    # Objectif : 
    model+= z  
    
    #Resolution du PLNE : 
    starttime=time.time() #Pour avoir le temps d'execution
    model.solve(PULP_CBC_CMD(maxSeconds=15))
    solveTime=time.time()-starttime
    
    #Affichage des éléments de résolution
    f_val=[]
    for i in range(0,len(Tasks)):
        f_val.append(model.variables()[i].varValue)
    order=np.argsort(f_val)
    
        
    print("Temps de résolution = ", solveTime)
    print("Statut de la solution = ", LpStatus[model.status])
    print("Valeur optimale = ", value(model.objective))
    print("Ordre des taches =",order)
    return(solveTime,value(model.objective),list(order))

In [13]:
solvePLNE(disp, dmax, duree)

Temps de résolution =  0.12602877616882324
Statut de la solution =  Infeasible
Valeur optimale =  99.827257
Ordre des taches = [ 0  1  7  8  9 10 11 12 13 14  2  3  4  5  6]


(0.12602877616882324,
 99.827257,
 array([ 0,  1,  7,  8,  9, 10, 11, 12, 13, 14,  2,  3,  4,  5,  6],
       dtype=int64))